In [ ]:
import pandas as pd
import numpy as np
import torch
import os

In [ ]:
import glob

In [ ]:
def process_csv_files(data_folder, output_file):
    # Get all CSV files in the data folder
    csv_files = glob.glob(os.path.join(data_folder, '*.csv'))
    
    # Function to read a CSV and add a simulation ID
    def read_csv_with_sim_id(file, sim_id):
        print(file)
        df = pd.read_csv(file)
        df['simulation_id'] = sim_id
        return df
    
    # Combine all CSV files into a single DataFrame, adding simulation IDs
    combined_df = pd.concat([read_csv_with_sim_id(f, i) for i, f in enumerate(csv_files)], ignore_index=True)
    
    # Sort the DataFrame by simulation_id, timestep, row, and column
    combined_df = combined_df.sort_values(['simulation_id', 'timestep', 'row', 'col'])
    
    # Group by simulation_id, row, and column
    grouped = combined_df.groupby(['simulation_id', 'row', 'col'])
    
    # Function to calculate deltas for a group
    def calculate_deltas(group):
        result = group.copy()
        result = result.sort_values(["timestep"])
        for column in group.columns:
            if column not in ['row', 'col', 'simulation_id']:
                result[f'{column}_next'] = group[column].shift(-1)
        return result
    
    # Apply the delta calculation to each group
    result_df = grouped.apply(calculate_deltas).reset_index(drop=True)
    
    # Sort the result by simulation_id, timestep, row, and column
    result_df = result_df.sort_values(['simulation_id', 'timestep', 'row', 'col'])
    
    # Save the result to a new CSV file
    result_df.to_csv(output_file, index=False)
    
    print(f"Processed data saved to {output_file}")

    return result_df

In [ ]:
# Usage
data_folder = '../../../data/pressure_free/'
output_file = 'combined_pressure_free_data_with_deltas.csv'
result_df = process_csv_files(data_folder, output_file)

In [ ]:
result_df[result_df.timestep == 0].head()

In [ ]:
result_df.simulation_id.value_counts()

In [ ]:
result_df.isna().sum()

In [ ]:
result_df = result_df.dropna()

In [ ]:
import pandas as pd
from collections import defaultdict

class Node:
    def __init__(self, sim_id, row, col, attributes):
        self.sim_id = sim_id
        self.row = row
        self.col = col
        self.attributes = attributes
        self.neighbors = []

    def add_neighbor(self, neighbor):
        self.neighbors.append(neighbor)

    def __repr__(self):
        return f"Node(sim_id={self.sim_id}, row={self.row}, col={self.col})"

class Graph:
    def __init__(self):
        self.nodes = {}

    def add_node(self, node):
        self.nodes[(node.sim_id, node.row, node.col)] = node

    def get_node(self, sim_id, row, col):
        return self.nodes.get((sim_id, row, col))

    def add_edge(self, node1, node2):
        node1.add_neighbor(node2)
        node2.add_neighbor(node1)

def df_to_graph(df):
    # Create a graph
    graph = Graph()
    
    # Group by simulation_id
    grouped = df.groupby('simulation_id')

    print(grouped.groups.keys())
    
    for sim_id, sim_data in grouped:
        print(sim_id)
        print(sim_data.shape)
        # First pass: Create nodes
        for _, row in sim_data.iterrows():
            if (_ % 5000) == 0:
                print(_)
            node = Node(sim_id, row['row'], row['col'], row.to_dict())
            graph.add_node(node)
        
        # Second pass: Create edges
        for _, row in sim_data.iterrows():
            current_node = graph.get_node(sim_id, row['row'], row['col'])
            
            # Define adjacent cells
            adjacent_cells = [
                (sim_id, row['row'], row['col'] + 1),  # right
                (sim_id, row['row'], row['col'] - 1),  # left
                (sim_id, row['row'] + 1, row['col']),  # down
                (sim_id, row['row'] - 1, row['col'])   # up
            ]
            
            # Add edges if adjacent cells exist
            for adj_cell in adjacent_cells:
                adj_node = graph.get_node(*adj_cell)
                
                if adj_node:
                    graph.add_edge(current_node, adj_node)
    
    return graph

In [ ]:
output_file = 'combined_data_with_deltas.csv'
results_df = pd.read_csv(output_file, nrows=40_000)

In [ ]:
# Usage example
graph = df_to_graph(results_df)

# Print some basic information about the graph
print(f"Number of nodes: {len(graph.nodes)}")

# Example of accessing node data
node_key = list(graph.nodes.keys())[0]  # Get the first node key
node = graph.nodes[node_key]
print(f"Data for node {node}:")
print(node.attributes)

# Example of iterating over neighbors
print(f"Neighbors of node {node}:")
for neighbor in node.neighbors:
    print(neighbor)

## Skipping the Previous Graph Structure to go Directly to Training Data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

# Check if MPS is available
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

class SimpleGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleGNN, self).__init__()
        self.embed = nn.Linear(input_dim, hidden_dim)
        self.message = nn.Linear(hidden_dim, hidden_dim)
        self.update = nn.Linear(2 * hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, adj_t):
        # Initial node embeddings
        h = F.relu(self.embed(x))
        
        # Message passing
        messages = torch.sparse.mm(adj_t, h)
        messages = F.relu(self.message(messages))
        
        # Update
        h_update = torch.cat([h, messages], dim=1)
        h = F.relu(self.update(h_update))
        
        # Output layer
        out = self.output(h)
        return out

class GraphDataset(Dataset):
    def __init__(self, csv_file, nrows=50000):
        self.df = pd.read_csv(csv_file, nrows=nrows)
        self.feature_cols = [col for col in self.df.columns if not col.endswith('_next') and col not in ['simulation_id', 'row', 'col']]
        self.target_cols = [col for col in self.df.columns if col.endswith('_next')]
        self.adj_t = self.create_sparse_adj_matrix()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        x = torch.tensor(self.df.iloc[idx][self.feature_cols].values, dtype=torch.float)
        y = torch.tensor(self.df.iloc[idx][self.target_cols].values, dtype=torch.float)
        return x, y

    def create_sparse_adj_matrix(self):
        num_nodes = len(self.df)
        row_indices = []
        col_indices = []
        
        for sim_id in self.df['simulation_id'].unique():
            sim_data = self.df[self.df['simulation_id'] == sim_id]
            for idx, row in sim_data.iterrows():
                current_node = (row['row'], row['col'])
                adjacent_cells = [
                    (row['row'], row['col'] + 1),  # right
                    (row['row'], row['col'] - 1),  # left
                    (row['row'] + 1, row['col']),  # down
                    (row['row'] - 1, row['col'])   # up
                ]
                for adj_cell in adjacent_cells:
                    adj_node = sim_data[(sim_data['row'] == adj_cell[0]) & (sim_data['col'] == adj_cell[1])]
                    if not adj_node.empty:
                        adj_idx = adj_node.index[0]
                        row_indices.extend([idx, adj_idx])
                        col_indices.extend([adj_idx, idx])
        
        edge_index = torch.tensor([row_indices, col_indices], dtype=torch.long)
        values = torch.ones(len(row_indices), dtype=torch.float)
        adj_t = torch.sparse_coo_tensor(edge_index, values, (num_nodes, num_nodes))
        return adj_t.t()

def train_gnn(model, train_loader, adj_matrix, num_epochs=20):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.MSELoss()
    
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            out = model(batch_x, adj_matrix)
            loss = criterion(out, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}')

In [ ]:
# Usage example
csv_file = 'combined_data_with_deltas.csv'
dataset = GraphDataset(csv_file, 100_000)

In [ ]:
device = "cpu"

In [ ]:
adj_t = dataset.adj_t.to(device)
batch_size = 32  # Adjust based on your dataset size and available memory
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
input_dim = len(dataset.feature_cols)
hidden_dim = 64
output_dim = len(dataset.target_cols)

model = SimpleGNN(input_dim, hidden_dim, output_dim).to(device)

# Train the model
train_gnn(model, train_loader, adj_t)

# Make predictions
model.eval()
all_predictions = []
with torch.no_grad():
    for batch_x, _ in train_loader:
        batch_x = batch_x.to(device)
        batch_predictions = model(batch_x, adj_matrix)
        all_predictions.append(batch_predictions.cpu())

predictions = torch.cat(all_predictions, dim=0)

print("Predictions shape:", predictions.shape)
print("First few predictions:", predictions[:5])

## Torch Geometric

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
import time

# Check if CUDA is available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device("mps")
print(f"Using device: {device}")

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.output(x)
        return x

class GraphDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, nrows=100_000, sample_frac=0.25):
        self.df = pd.read_csv(csv_file, nrows=nrows)
        print(self.df.shape)
        row_max = self.df.row.max()
        self.df.loc[:, "border"] = 0.0
        col_max = self.df.col.max()
        # TODO: we probably still can subsample but we need to throw out any cell without 3+ neighbors
        self.df = self.df[(self.df.simulation_id == 0) & (self.df.timestep == 3)]
        print(self.df.shape)
        self.df.loc[self.df["row"].isin([0, row_max]) | self.df["col"].isin([0, col_max]), "border"] = 1.0
        self.feature_cols = [col for col in self.df.columns if not col.endswith('_next') and col not in ['simulation_id', 'row', 'col']]
        self.target_cols = [col for col in self.df.columns if col.endswith('_next')]
        self.simulations = self.df['simulation_id'].unique()
        
        # Precompute edge_indices for all simulations
        print('precompute_edge_indices')
        self.edge_indices = self.precompute_edge_indices()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sim_data = self.df.iloc[idx]

        sim_id = sim_data.simulation_id
        timestep = sim_data.timestep
        
        x = torch.tensor(sim_data[self.feature_cols].values, dtype=torch.float)
        y = torch.tensor(sim_data[self.target_cols].values, dtype=torch.float)
        
        # Use the precomputed edge_index
        edge_index = self.edge_indices[sim_id][timestep]

        print(f'edge index: {sim_id} {timestep}')
        print(edge_index)
        
        return Data(x=x, edge_index=edge_index, y=y)

    def precompute_edge_indices(self):
        start = time.time()
        edge_indices = {}
        for sim_id in self.simulations:
            edge_indices[sim_id] = {}
            sim_data = self.df[self.df['simulation_id'] == sim_id]
            print(f'Timesteps: {ist(sim_data.timestep.unique()}')
            for timestep in list(sim_data.timestep.unique()):
                sim_data_timestep = sim_data[sim_data.timestep == timestep]
                edge_indices[sim_id][timestep] = self.create_edge_index(sim_data_timestep)
        print(f'{time.time() - start}')
        return edge_indices

    def create_edge_index(self, sim_data):
        edges = []
        for idx, row in sim_data.iterrows():
            if idx % 2_000 == 0:
                print(idx)
            current_node = (row['row'], row['col'])
            adjacent_cells = [
                (row['row'], row['col'] + 1),  # right
                (row['row'], row['col'] - 1),  # left
                (row['row'] + 1, row['col']),  # down
                (row['row'] - 1, row['col'])   # up
            ]
            for adj_cell in adjacent_cells:
                adj_node = sim_data[
                    (sim_data['row'] == adj_cell[0]) & 
                    (sim_data['col'] == adj_cell[1]) #& (sim_data["timestep"] == row["timestep"])
                ]
                if not adj_node.empty:
                    adj_idx = adj_node.index[0]
                    edges.append([sim_data.index.get_loc(idx), sim_data.index.get_loc(adj_idx)])
        
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        return edge_index


def train_gnn(model, train_loader, num_epochs=80):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.MSELoss()
    
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index)
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}')

In [ ]:
# csv_file = 'combined_data_with_deltas.csv'
# data = pd.read_csv(csv_file, nrows=500_000).sample(frac=0.2)

In [ ]:
# data.head()

In [ ]:
# Usage example
csv_file = 'combined_data_with_deltas.csv'
dataset = GraphDataset(csv_file, 2_500_000, 0.005)

In [ ]:
print(device)

In [ ]:
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

input_dim = len(dataset.feature_cols)
hidden_dim = 128
output_dim = len(dataset.target_cols)

model = GCN(input_dim, hidden_dim, output_dim).to(device)

# Train the model
# train_gnn(model, train_loader)

# Make predictions
model.eval()
all_predictions = []
with torch.no_grad():
    for batch in train_loader:
        batch = batch.to(device)
        # print(batch.edge_index)
        print(batch.x.shape)
        print(batch.edge_index.shape)
        batch_predictions = model(batch.x, batch.edge_index)
        all_predictions.append(batch_predictions.cpu())
        all_predictions.append(batch)

predictions = torch.cat(all_predictions, dim=0)

print("Predictions shape:", predictions.shape)
print("First few predictions:", predictions[:5])